In [3]:
import json
import pickle
import collections
from tqdm.notebook import tqdm
import re

In [4]:
vocab = collections.defaultdict(int)

f = open('/home/ubuntu/recipe-dataset/json/cleaned_layers.json', 'r')
data = json.load(f)

In [5]:
for id in tqdm(data, total=len(data)):
    recipe = data[id]

    title = re.sub(r'[^\w\s]',' ',recipe['title'].lower())
    for word in title.split(' '):
        vocab[word] += 1
    
    for instruction in recipe['instructions']:
        instruction = re.sub(r'[^\w\s]',' ',instruction.lower())
        for word in instruction.split(' '):
            vocab[word] += 1
    
    for ingredient in recipe['ingredients']:
        ingredient = re.sub(r'[^\w\s]',' ',ingredient.lower())
        for word in ingredient.split(' '):
            vocab[word] += 1


  0%|          | 0/1029720 [00:00<?, ?it/s]

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
from transformers import BertTokenizer, BertModel
from models.bert_encoder import BERTEncoder
bert_encoder = BERTEncoder(BertModel.from_pretrained('bert-base-uncased'), BertTokenizer.from_pretrained('bert-base-uncased'), device=device)

bert_encoder.run('/home/ubuntu/recipe-dataset/json/vocab.pkl', '/home/ubuntu/recipe-dataset/json/vocab_bert.pkl')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of words in vocabulary: 151344
Creating embeddings...


100%|██████████| 1514/1514 [01:12<00:00, 20.86it/s]


Embeddings saved to /home/ubuntu/recipe-dataset/json/vocab_bert.pkl


In [21]:
import numpy as np

partition = 'test'
ids = pickle.load(open('/home/ubuntu/recipe-dataset/test/test_keys.pkl', 'rb'))
data = json.load(open('/home/ubuntu/recipe-dataset/json/cleaned_layers.json', 'r'))
new_data = {}
for i, (id, sample) in enumerate(data.items()):
    if sample['partition'] == partition:
        new_data[id] = sample

data = new_data
del new_data

image_map = json.load(open('/home/ubuntu/recipe-dataset/json/image_map.json', 'r'))

torch.random.manual_seed(42)
np.random.seed(42)
random_embedding = torch.randn(768)

bert_embeddings = pickle.load(open('/home/ubuntu/recipe-dataset/json/vocab_bert.pkl', 'rb'))
bert_embeddings = {k: torch.tensor(v) for k, v in bert_embeddings.items()}
ingredient_vocabulary = pickle.load(open('/home/ubuntu/recipe-dataset/json/ingredient_vocab.pkl', 'rb'))

In [22]:
id = 5
sample = data[ids[id]]

image_ids = image_map[ids[id]]
image_id = np.random.choice(image_ids)
dataset_images = '/home/ubuntu/recipe-dataset/test/'
image_path = dataset_images +'/'.join(list(image_id[:4])) + '/' + image_id

title = sample['title']
ingredients = sample['ingredients']
instructions = sample['instructions']

In [ ]:
title_embedding = [bert_embeddings.get(word, random_embedding) for word in title.lower().split(' ')]

instruction_embedding = []
for instruction in instructions:
    temp = []
    instruction = re.sub(r"[^a-zA-Z0-9]", " ", instruction.lower())
    print(f"instruction: {instruction}")
    for word in instruction.strip().split(' '):
        e = bert_embeddings.get(word, random_embedding)
        temp.append(e)
    instruction_embedding.append(torch.cat(temp))

# ingredient embeddings contain an additional lookup in the ingredient vocabulary
ingredient_embedding = []
for ingredient in ingredients:
    temp = []
    ingredient = re.sub(r"[^a-zA-Z0-9]", " ", ingredient.lower())
    for word in ingredient.split(" "):
        if word in ingredient_vocabulary['ingredient2stem']:
            temp.append(bert_embeddings.get(word, random_embedding))
    
    ingredient_embedding.append(torch.cat(temp))

In [ ]:
ingredient_vocabulary['ingredient2stem']
